In [1]:
import pandas as pd
import numpy as np
import re
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('climate_data_final.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'climate_data_final.csv'

In [ ]:
df.shape

In [ ]:
df.state = df.state.str.strip()
print(df.state.unique())
print(len(df.state.unique()))

In [ ]:
df.humidity[:5].tolist()

In [ ]:
doct = df.drop('city', axis=1)

In [ ]:
doct['A'] = doct['humidity'].str.replace('The|number|rainy|month|with|the|highest|relative|humidity|is|\
                                        |lowest|which|sess|most|rainfall|days|of|least|%|while|year|that|\
                                        |sees|amount|percent|\n|wettest|driest|whilst|fewest|(|)', '', 
                                         regex=True)
doct = doct.drop('humidity', axis=1)
doct.head()

In [ ]:
doct.A[280:300].tolist()

In [ ]:
doct = doct["A"].str.split(n=11, expand=True)
doct.columns = ['Humidity_{}'.format(x+1) for x in doct.columns]
doct

In [ ]:
clean_weath = doct[['Humidity_2', 'Humidity_5', 'Humidity_8', 'Humidity_11']]
clean_weath['Humidity_2'] = clean_weath['Humidity_2'].str.extract(r'(\d+.\d+)').astype('float')
clean_weath['Humidity_5'] = clean_weath['Humidity_5'].str.extract(r'(\d+.\d+)').astype('float')
clean_weath['Humidity_8'] = clean_weath['Humidity_8'].str.extract(r'(\d+.\d+)').astype('float')
clean_weath['Humidity_11'] = clean_weath['Humidity_11'].str.extract(r'(\d+.\d+)').astype('float')
clean_weath

In [ ]:
new_df = pd.concat([df, clean_weath], axis=1)

In [ ]:
new_df

In [ ]:
state_df = new_df.drop('humidity', axis=1)
state_df.head()

In [ ]:
# state_df['rainfall'] = state_df['rainfall'].str.replace(r'([a-zA-Z|\n/.,>:()'']+)', '', regex=True)
state_df['rainfall'] = state_df['rainfall'].str.extract(r'(\d+.\d+)').astype('float')

In [ ]:
state_df.info()

In [ ]:
state_df['mean_humidity'] = state_df[['Humidity_2', 'Humidity_5']].mean(axis=1)
state_df['mean_rainfall'] = state_df[['Humidity_8', 'Humidity_11']].mean(axis=1)
state_df_ = state_df.drop(columns=['Humidity_2', 'Humidity_5', 'Humidity_8', 'Humidity_11', 'city'])
state_df_

In [ ]:
state_mean_df = state_df_.groupby(['state']).mean()
state_mean_df.reset_index(level=0, inplace=True)
state_mean_df

In [ ]:
soil_df = pd.read_csv('Nigeria_soils_ocp.csv')
soil_df['state'] = soil_df['state'].str.replace('State', '', regex=True)
soil_df.head()

In [ ]:
soil_df.shape

In [ ]:
soil_df.state = soil_df.state.str.strip()
soil_df.state.value_counts()

In [ ]:
final_df = pd.merge(state_mean_df, soil_df, on='state')
final_df.head()

In [ ]:
final_df.shape

In [ ]:
crop_list = final_df['MAJOR_CROP'].str.split(',')
final_df.MAJOR_CROP = crop_list
final_df = final_df.explode('MAJOR_CROP').reset_index(drop=True)
final_df.head()

In [ ]:
final_df.shape

In [ ]:
final_df['Depth'] = final_df['Depth'].str.replace('cm', '', regex=True)
final_df.Depth = final_df.Depth.str.replace(" ", '')
final_df.SOIL_PH = final_df.SOIL_PH.str.replace(" ", '')
final_df.SLOPE = final_df.SLOPE.str.replace(" ", '')
final_df['SLOPE'] = final_df['SLOPE'].str.replace('%', '', regex=True)
final_df[['Depth1', 'Depth2']] = final_df['Depth'].str.split('-|–', expand=True)
final_df[['SOIL_PH1', 'SOIL_PH2']] = final_df['SOIL_PH'].str.split('-', expand=True)
final_df[['col', 'slope']] = final_df['SLOPE'].str.split('-', expand=True)

final_df = final_df.drop(columns=['Depth', 'SOIL_PH', 'Depthcode', 'col', 'SLOPE'])
final_df

In [ ]:
final_df.Depth1 = final_df.Depth1.astype(float)
final_df.Depth2 = final_df.Depth2.astype(float)
final_df.SOIL_PH1 = final_df.SOIL_PH1.astype(float)
final_df.SOIL_PH2 = final_df['SOIL_PH2'].str.extract(r'(\d+.\d+)').astype('float')
final_df.slope = final_df.slope.astype(float)

In [ ]:
final_df['avg_depth'] = final_df[['Depth1', 'Depth2']].mean(axis=1)
final_df['avg_soil_ph'] = final_df[['SOIL_PH1', 'SOIL_PH1']].mean(axis=1)

final_df = final_df.drop(columns=['Depth1', 'Depth2', 'SOIL_PH1', 'SOIL_PH2', 'city', 'project', 'ID', 
                                'project', 'MAPPING_UN', 'GEOLOGY', 'ECOLOGICAL', 'DRAINAGE', 'PH_DESCRIP', 
                                'SUITABILIT', 'SOIL_TEXTU', 'DISTRIBUTI', 'PERCENTAGE'])

In [ ]:
final_df

In [ ]:
final_df.MAJOR_CROP.value_counts()

In [ ]:
crop_df = final_df[final_df['MAJOR_CROP'].isin(['Yam', 'Maize', 'Sorghum', 'Cotton', 'Cassava', 'Groundnut',
                                                'Cocoa', 'Rice', 'Irish Potatoes', 'Cowpea', 'Millet',
                                                ' Millet', ' Rice', 'Cow Pea', 'Vegetables', 'Bean', 'Beans',
                                                'G.nuts', 'Upland rice', 'Cocoa yam', ' Groundnut', ' Yam',
                                                ' Maize', ' Cowpea', 'Maize.Cocoa', 'Vegetable', 'Millets'])]
crop_df

In [ ]:
# y = crop_df['MAJOR_CROP']
# X = crop_df.drop(columns=['MAJOR_CROP'], axis=1)

In [ ]:
crop_df.shape

In [ ]:
final_crop = crop_df.drop(columns=['county', 'SOIL_CLASS', 'SOIL_CLA_1', 'Depth_2'])
final_crop = final_crop.reset_index()
final_crop

In [ ]:
vect = CountVectorizer()

vect_crop = vect.fit_transform(final_crop)
print("Count Matrix:", vect_crop.toarray())

In [ ]:
cosine_sim = cosine_similarity(vect_crop)

In [ ]:
crops_to_plant = "Rice"

def get_index_from_title(crop):
    return final_crop[final_crop.MAJOR_CROP == crop]["index"].values[0]

movie_index = get_index_from_title(crops_to_plant)

In [ ]:
similar_crops = list(enumerate(cosine_sim[movie_index]))

In [ ]:
sorted_similar_crop = sorted(similar_crops, key=lambda x:x[1], reverse=True)

In [ ]:
def get_title_from_index(index):
    return final_crop[final_crop.index == index]["MAJOR_CROP"].values[0]
i=0
for crop in sorted_similar_crop:
    print(get_title_from_index(crop[0]))
    i=i+1
    if i>5:
        break

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,stratify=y, random_state = 42)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

## Converting categorical variables to categorical types

In [ ]:
X_train['state'] = X_train['state'].astype('category')
X_train['VEGETATION'] = X_train['VEGETATION'].astype('category')
X_train['Farm'] = X_train['Farm'].astype('category')

In [ ]:
y_train.value_counts()

## class balancing using SMOTENC model

In [ ]:
smotenc = SMOTENC(categorical_features=[0, 5, 18],random_state = 101, sampling_strategy='minority', k_neighbors=1)
X_oversample, y_oversample = smotenc.fit_resample(X_train, y_train)

In [ ]:
y_oversample.value_counts()

In [ ]:
X_oversample_ = X_oversample.drop(columns=['state', 'Farm', 'VEGETATION'])

## Feature Selection

In [ ]:
sel_df = SelectKBest(chi2, k=19,)

x_train = sel_df.fit_transform(X_oversample_, y_oversample)
df = pd.DataFrame(x_train)
df.head()